# Import Libraries

In [ ]:
#DataFrames manipulation
import pandas as pd

#Library to calculate the percentiles
from scipy import stats

#Data download from Yahoo Finance
from yahooquery import Ticker

#Libraries for the Plotting
import holoviews as hv
from holoviews import opts, dim
from bokeh.models import HoverTool
from holoviews.plotting.links import RangeToolLink

#Librarie to save the plots to html object
import panel as pn

#Quandl to take data from Multipl
import quandl

hv.extension('bokeh')

# Download S&P Dow Jones Indices Data

In [ ]:
#Download the data from S&P Dow Jones website. Only need the data from tab "Quarterly Data"
data = pd.read_excel('https://www.spglobal.com/spdji/en/documents/additional-material/sp-500-eps-est.xlsx', sheet_name='QUARTERLY DATA')

In [ ]:
#Define the columns
data.columns = ['Date', 'Op_EPS', 'EPS', 'DPS', 'SPS','BVPS','CAPPS','Price','Divisor']

#Limit the DataFrame just for when there is data
data = data[6:]

#Change 'Date' column to datetime format
data['Date'] = pd.to_datetime(data['Date'])

#Limit the Dataframe just for the necessary columns
data = data[['Date', 'EPS', 'DPS', 'Price']]

#Sort the Dataframe
data = data.sort_values(by=['Date'])

#Get the trailing annual values for the EPS and DPS - Sum of last 4 quarters
data['EPS'] = data['EPS'].rolling(window=4).sum()
data['DPS'] = data['DPS'].rolling(window=4).sum()
data = data.dropna()

#Limit the Dataframe to have only December Values - Annual base
data = data[data['Date'].dt.month == 12]

In [ ]:
#Create a New Object
final_data = data

#Calculate the EPS Growth and 
final_data['EPS_Growth'] = round(final_data['EPS'].pct_change() * 100)

#Calculate the Dividend Yield
final_data['DPS'] = pd.to_numeric(final_data['DPS'], errors='coerce')
final_data['Price'] = pd.to_numeric(final_data['Price'], errors='coerce')
final_data['Div_Yield'] = round(final_data['DPS'] / final_data['Price'] * 100, 2)

#Calculate the P/E ratio change
final_data['PE_ratio'] = round(final_data['Price'] / final_data['EPS'], 2)
final_data['PE_ratio_change'] = round(final_data['PE_ratio'].pct_change() * 100)

#Get the Year from the Date
final_data['Year'] = final_data['Date'].dt.year

#Limit the Dataframe to have only the Necessary Columns and drop NA's
final_data = final_data[['Year', 'Div_Yield', 'EPS_Growth', 'PE_ratio_change']]
final_data = final_data.dropna()

# Download S&P 500 Total Return Data from Yahoo Finance 

In [ ]:
#Download the Data from Yahoo Finance
tickers = Ticker('^SP500TR')
spx_total_returns = tickers.history(start = '1970-01-01')

#Limit the Data to the Close and rearrange columns
spx_total_returns = spx_total_returns.reset_index()
spx_total_returns = spx_total_returns[['date', 'close']]
spx_total_returns = spx_total_returns.rename(columns = {'date': 'Date', 'close': 'SP500_Total_Return'})

# Convert the 'Date' column to datetime format 
spx_total_returns["Date"] = pd.to_datetime(spx_total_returns["Date"], utc = 'true')

#Limit the Dataframe to get only the last day of each year
spx_total_returns = spx_total_returns.groupby(spx_total_returns['Date'].dt.year).last()

#Calculate the annual returns
spx_total_returns['SP500_Total_Return'] = round(spx_total_returns['SP500_Total_Return'].pct_change() * 100, 2)

#Get the Year from the Date
spx_total_returns['Year'] = spx_total_returns['Date'].dt.year

#Drop NA's and Keep only the Year and the Returns columns
spx_total_returns = spx_total_returns[['Year', 'SP500_Total_Return']]

# Merge the Dataframes and Create the Graph

In [ ]:
#Merge the Data dataframe with the spx_total_returns dataframe
full_data = final_data.merge(spx_total_returns, on='Year', how='left')

In [ ]:
#Stack the data to create the Stacked barchart
stacked_data = full_data.melt(id_vars=['Year'], value_vars=['Div_Yield', 'EPS_Growth', 'PE_ratio_change'])

#Create the stacked barchart
stacked_barchart = hv.Bars(stacked_data, ['Year', 'variable'], 'value').opts(
    stacked=True, width=1000, height=400, ylabel='Variables (%)', tools=['hover'])

#Line chart
linechart = hv.Curve(full_data[['Year', 'SP500_Total_Return']]).opts(
    color='black', line_width=2, width=1000, height=400, tools=['hover'])

#Overlay Both Charts
full_chart = (stacked_barchart * linechart).opts(legend_position='top_left', title = 'S&P 500 Index: Annual Total Returns Decomposition')

In [ ]:
#Save the Plot
p = pn.panel(full_chart)
p.save('Market_Returns_Decomposition_chart.html', embed = True)

# Manipulate the Data to get the chart for 5 Year Returns

In [ ]:
#Create a New Object
final_data_5Y = data.set_index('Date').resample('5Y').last()

#Reset Index
final_data_5Y = final_data_5Y.reset_index()

#Calculate the EPS Growth and 
final_data_5Y['EPS_Growth'] = round(final_data_5Y['EPS'].pct_change() * 100)

#Calculate the Dividend Yield
final_data_5Y['DPS'] = pd.to_numeric(final_data_5Y['DPS'], errors='coerce')
final_data_5Y['Price'] = pd.to_numeric(final_data_5Y['Price'], errors='coerce')
final_data_5Y['Div_Yield'] = round(final_data_5Y['DPS'] / final_data_5Y['Price'] * 100, 2)

#Calculate the P/E ratio change
final_data_5Y['PE_ratio'] = round(final_data_5Y['Price'] / final_data_5Y['EPS'], 2)
final_data_5Y['PE_ratio_change'] = round(final_data_5Y['PE_ratio'].pct_change() * 100)

#Get the Year from the Date
final_data_5Y['Year'] = final_data_5Y['Date'].dt.year - 1

#Limit the Dataframe to have only the Necessary Columns and drop NA's
final_data_5Y = final_data_5Y[['Year', 'Div_Yield', 'EPS_Growth', 'PE_ratio_change']]
final_data_5Y = final_data_5Y.dropna()

#Get the 5-year average for the EPS_Growth and PE_ratio change
final_data_5Y['EPS_Growth'] = final_data_5Y['EPS_Growth']/5
final_data_5Y['PE_ratio_change'] = final_data_5Y['PE_ratio_change']/5

#Dividend Yield based on the 5 years average
final_data_5Y['Div_Yield'] = data[['Date','Div_Yield']].set_index('Date').resample('5Y').sum().reset_index()[1:]['Div_Yield']/5

In [ ]:
#Download the Data from Yahoo Finance
tickers = Ticker('^SP500TR')
spx_total_returns_5Y = tickers.history(start = '1988-01-01', end = '2022-12-31')

#Limit the Data to the Close and rearrange columns
spx_total_returns_5Y = spx_total_returns_5Y.reset_index()
spx_total_returns_5Y = spx_total_returns_5Y[['date', 'close']]
spx_total_returns_5Y = spx_total_returns_5Y.rename(columns = {'date': 'Date', 'close': 'SP500_Total_Return'})

# Convert the 'Date' column to datetime format 
spx_total_returns_5Y["Date"] = pd.to_datetime(spx_total_returns_5Y["Date"], utc = 'true')

#Resample the Dataframe to 5Years
spx_total_returns_5Y = spx_total_returns_5Y.set_index('Date')
spx_total_returns_5Y = spx_total_returns_5Y.resample('5Y').last()
spx_total_returns_5Y = spx_total_returns_5Y.reset_index()

#Limit the Dataframe to get only the last day of each year
spx_total_returns_5Y = spx_total_returns_5Y.groupby(spx_total_returns_5Y['Date'].dt.year).last()

#Calculate the annual returns
spx_total_returns_5Y['SP500_Total_Return'] = round(spx_total_returns_5Y['SP500_Total_Return'].pct_change() * 100, 2)

#Get the Year from the Date
spx_total_returns_5Y['Year'] = spx_total_returns_5Y['Date'].dt.year

#Drop NA's and Keep only the Year and the Returns columns
spx_total_returns_5Y = spx_total_returns_5Y[['Year', 'SP500_Total_Return']]
spx_total_returns_5Y = spx_total_returns_5Y.dropna()

#Get the Average Yearly Return by dividing by 5
spx_total_returns_5Y['SP500_Total_Return'] = spx_total_returns_5Y['SP500_Total_Return']/5

#Correct the Year
spx_total_returns_5Y['Year'] = spx_total_returns_5Y['Year']-1

In [ ]:
#Merge the Data dataframe with the spx_total_returns dataframe
full_data_5Y = final_data_5Y.merge(spx_total_returns_5Y, on='Year', how='left')

In [ ]:
#Stack the data to create the Stacked barchart
stacked_data_5Y = full_data_5Y.melt(id_vars=['Year'], value_vars=['Div_Yield', 'EPS_Growth', 'PE_ratio_change'])

#Create the stacked barchart
stacked_barchart_5Y = hv.Bars(stacked_data_5Y, ['Year', 'variable'], 'value').opts(
    stacked=True, width=1000, height=400, ylabel='Variables (%)', tools=['hover'])

#Line chart
linechart_5Y = hv.Curve(full_data_5Y[['Year', 'SP500_Total_Return']]).opts(
    color='black', line_width=2, width=1000, height=400, tools=['hover'])

#Overlay Both Charts
full_chart_5Y = (stacked_barchart_5Y * linechart_5Y).opts(legend_position='top_left', title = 'S&P 500 Index: Average 5 Year Total Returns Decomposition')

In [ ]:
#Save the Plot
p = pn.panel(full_chart_5Y)
p.save('Market_5Year_Avg_Returns_Decomposition_chart.html', embed = True)

# Historical P/E Ratio and Dividend Yield Graph

In [ ]:
#Quandl API key - Created a Free Account
quandl.ApiConfig.api_key = "e6Ex3wh4yNSyuUEGh75V"

In [ ]:
#S&P 500 PE Ratio Data
sp_500_pe_ratio = quandl.get("MULTPL/SP500_PE_RATIO_MONTH")
sp_500_pe_ratio = sp_500_pe_ratio.rename(columns={'Value': 'SP500_PE_ratio'})
sp_500_pe_ratio = sp_500_pe_ratio.resample('M').last()

#S&P 500 Dividend Yield Data
sp_500_div_yield = quandl.get("MULTPL/SP500_DIV_YIELD_MONTH")
sp_500_div_yield = sp_500_div_yield.rename(columns={'Value': 'SP500_Div_Yield'})

In [ ]:
#Calculate the Median and Percentile for the PE ratio Dataframe
sp_500_pe_ratio['Median'] = round(sp_500_pe_ratio['SP500_PE_ratio'].median(), 2)
sp_500_pe_ratio['90th_percentile'] = round(sp_500_pe_ratio['SP500_PE_ratio'].quantile(0.9), 2)

#Calculate the Median and Percentile for the Dividend Yield Dataframe
sp_500_div_yield['Median'] = round(sp_500_div_yield['SP500_Div_Yield'].median(), 2)
sp_500_div_yield['10th_percentile'] = round(sp_500_div_yield['SP500_Div_Yield'].quantile(0.1), 2)

In [ ]:
#Calculate the Percentiles
sp_500_pe_ratio_percentile = stats.percentileofscore(sp_500_pe_ratio['SP500_PE_ratio'], sp_500_pe_ratio['SP500_PE_ratio'].iloc[-1])
sp_500_div_yield_percentile = stats.percentileofscore(sp_500_div_yield['SP500_Div_Yield'], sp_500_div_yield['SP500_Div_Yield'].iloc[-1])

print(round(sp_500_pe_ratio_percentile,0))
print(round(sp_500_div_yield_percentile,0))

In [ ]:
#Instantiate the Dataframe for the Graph
graph_df = sp_500_pe_ratio

#Generate all curves
def getCurves(n):
    for column in graph_df.columns:
        hover = HoverTool(tooltips=[("Date", "@Date{%F}"), (column, f"@{column+'{0,.00}'}")], formatters={'@Date': 'datetime'})  
        curve = hv.Curve(graph_df[column], label = column).opts(opts.Curve(tools=[hover]))
        curve = curve.opts(xticks=10)
        yield curve
        
source_curves, target_curves  = [], []
for curve in getCurves(2):
    
    src = curve.relabel('').opts(width=800, height=100, yaxis=None) 
    tgt = curve.opts(width=800, ylabel = 'P/E Ratio')
    source_curves.append(src)
    target_curves.append(tgt)

# Link RangeTool for the first curves in the list.
RangeToolLink(source_curves[0],target_curves[0], axes=['x','y'])  

#Overlay the source and target curves 
overlaid_plot_src = hv.Overlay(source_curves).relabel('')
overlaid_plot_tgt = hv.Overlay(target_curves)

overlaid_plot_tgt = overlaid_plot_tgt.relabel('S&P 500: P/E Ratio History').opts(
    height=400, legend_position='top')

# Layout the plot
full_graph = (overlaid_plot_tgt + overlaid_plot_src).cols(1)
full_graph = full_graph.opts(merge_tools=False, shared_axes=False)

In [ ]:
#Save the Plots
p = pn.panel(full_graph)
p.save('PE_ratio_decomposing_stock_returns_chart.html', embed = True)

In [ ]:
#Instantiate the Dataframe for the Graph
graph_df = sp_500_div_yield

#Generate all curves
def getCurves(n):
    for column in graph_df.columns:
        hover = HoverTool(tooltips=[("Date", "@Date{%F}"), (column, f"@{column+'{0,.00}'}")], formatters={'@Date': 'datetime'})  
        curve = hv.Curve(graph_df[column], label = column).opts(opts.Curve(tools=[hover]))
        curve = curve.opts(xticks=10)
        yield curve
        
source_curves, target_curves  = [], []
for curve in getCurves(2):
    
    src = curve.relabel('').opts(width=800, height=100, yaxis=None) 
    tgt = curve.opts(width=800, ylabel = 'Dividend Yield (%)')
    source_curves.append(src)
    target_curves.append(tgt)

# Link RangeTool for the first curves in the list.
RangeToolLink(source_curves[0],target_curves[0], axes=['x','y'])  

#Overlay the source and target curves 
overlaid_plot_src = hv.Overlay(source_curves).relabel('')
overlaid_plot_tgt = hv.Overlay(target_curves)

overlaid_plot_tgt = overlaid_plot_tgt.relabel('S&P 500: Dividend Yield (%) History').opts(
    height=400, legend_position='top')

# Layout the plot
full_graph = (overlaid_plot_tgt + overlaid_plot_src).cols(1)
full_graph = full_graph.opts(merge_tools=False, shared_axes=False)

In [ ]:
#Save the Plots
p = pn.panel(full_graph)
p.save('Div_Yield_decomposing_stock_returns_chart.html', embed = True)